# Steam游戏推荐系统 - 数据探索

这个笔记本用于探索Steam游戏数据集，了解数据结构和特征分布，为构建推荐系统做准备。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sys
import os

# 添加项目根目录到路径
sys.path.append('..')

# 设置matplotlib样式
plt.style.use('seaborn-v0_8-whitegrid')

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

## 1. 加载数据

In [ ]:
# 数据路径
data_path = "../data"

# 读取CSV文件
games_df = pd.read_csv(f"{data_path}/games.csv")
users_df = pd.read_csv(f"{data_path}/users.csv")
recommendations_df = pd.read_csv(f"{data_path}/recommendations.csv")

# 读取JSON元数据
metadata_list = []
with open(f"{data_path}/games_metadata.json", 'r', encoding='utf-8') as f:
    for line in f:
        try:
            metadata_list.append(json.loads(line))
        except json.JSONDecodeError:
            continue

metadata_df = pd.DataFrame(metadata_list)

print(f"已加载 {len(games_df)} 个游戏")
print(f"已加载 {len(users_df)} 个用户")
print(f"已加载 {len(recommendations_df)} 条评价")
print(f"已加载 {len(metadata_df)} 条游戏元数据")

## 2. 探索游戏数据

In [ ]:
# 查看游戏数据的前几行
games_df.head()

In [ ]:
# 查看数据结构
games_df.info()

In [ ]:
# 查看数据统计摘要
games_df.describe(include='all')

In [ ]:
# 分析游戏发布年份分布
games_df['year'] = pd.to_datetime(games_df['date_release']).dt.year

plt.figure(figsize=(12, 6))
year_counts = games_df['year'].value_counts().sort_index()
year_counts.plot(kind='bar')
plt.title('游戏发布年份分布')
plt.xlabel('发布年份')
plt.ylabel('游戏数量')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 分析游戏评分分布
plt.figure(figsize=(10, 6))
sns.histplot(games_df['positive_ratio'], bins=20)
plt.title('游戏好评率分布')
plt.xlabel('好评率 (%)')
plt.ylabel('游戏数量')
plt.tight_layout()
plt.show()

In [ ]:
# 分析游戏支持的平台
platform_data = {
    'Windows': games_df['win'].sum(),
    'Mac': games_df['mac'].sum(),
    'Linux': games_df['linux'].sum()
}

plt.figure(figsize=(8, 5))
plt.bar(platform_data.keys(), platform_data.values())
plt.title('游戏平台支持情况')
plt.xlabel('平台')
plt.ylabel('游戏数量')
for i, v in enumerate(platform_data.values()):
    plt.text(i, v + 50, f"{v} ({v/len(games_df):.1%})", ha='center')
plt.tight_layout()
plt.show()

## 3. 探索游戏元数据

In [ ]:
# 查看元数据的前几行
metadata_df.head()

In [ ]:
# 分析游戏标签
# 展开标签列表
all_tags = []
for tags in metadata_df['tags']:
    if isinstance(tags, list):
        all_tags.extend(tags)

# 计算标签频率
tag_counts = pd.Series(all_tags).value_counts()

# 显示前20个最常见的标签
plt.figure(figsize=(12, 8))
tag_counts[:20].plot(kind='barh')
plt.title('前20个最常见的游戏标签')
plt.xlabel('出现次数')
plt.ylabel('标签')
plt.tight_layout()
plt.show()

In [ ]:
# 分析描述长度
metadata_df['description_length'] = metadata_df['description'].apply(lambda x: len(x) if isinstance(x, str) else 0)

plt.figure(figsize=(10, 6))
sns.histplot(metadata_df['description_length'], bins=50)
plt.title('游戏描述长度分布')
plt.xlabel('描述长度 (字符数)')
plt.ylabel('游戏数量')
plt.xlim(0, 2000)  # 限制x轴范围以便更好地查看分布
plt.tight_layout()
plt.show()

## 4. 探索用户数据

In [ ]:
# 查看用户数据的前几行
users_df.head()

In [ ]:
# 分析用户拥有的游戏数量分布
plt.figure(figsize=(10, 6))
sns.histplot(users_df['products'], bins=50)
plt.title('用户拥有的游戏数量分布')
plt.xlabel('拥有游戏数量')
plt.ylabel('用户数量')
plt.xlim(0, 500)  # 限制x轴范围以便更好地查看分布
plt.tight_layout()
plt.show()

In [ ]:
# 分析用户评价数量分布
plt.figure(figsize=(10, 6))
sns.histplot(users_df['reviews'], bins=20)
plt.title('用户评价数量分布')
plt.xlabel('评价数量')
plt.ylabel('用户数量')
plt.tight_layout()
plt.show()

## 5. 探索评价数据

In [ ]:
# 查看评价数据的前几行
recommendations_df.head()

In [ ]:
# 分析游戏时长分布
plt.figure(figsize=(10, 6))
sns.histplot(recommendations_df['hours'], bins=50)
plt.title('游戏时长分布')
plt.xlabel('游戏时长 (小时)')
plt.ylabel('评价数量')
plt.xlim(0, 200)  # 限制x轴范围以便更好地查看分布
plt.tight_layout()
plt.show()

In [ ]:
# 分析评价是否推荐的比例
rec_counts = recommendations_df['is_recommended'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(rec_counts, labels=['推荐', '不推荐'], 
        autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#ff9999'])
plt.title('游戏评价推荐比例')
plt.tight_layout()
plt.show()

In [ ]:
# 分析游戏时长与评价的关系
plt.figure(figsize=(10, 6))
sns.boxplot(x='is_recommended', y='hours', data=recommendations_df)
plt.title('游戏时长与评价关系')
plt.xlabel('是否推荐')
plt.ylabel('游戏时长 (小时)')
plt.ylim(0, 100)  # 限制y轴范围以便更好地查看分布
plt.tight_layout()
plt.show()

## 6. 数据集交叉分析

In [ ]:
# 合并游戏和评价数据
game_rec_df = pd.merge(recommendations_df, games_df, on='app_id')

In [ ]:
# 分析不同评分游戏的评价情况
plt.figure(figsize=(12, 6))
sns.barplot(x='rating', y='is_recommended', data=game_rec_df)
plt.title('游戏评分与用户推荐率的关系')
plt.xlabel('游戏评分')
plt.ylabel('推荐比例')
plt.tight_layout()
plt.show()

In [ ]:
# 分析用户-游戏交互稀疏程度
unique_users = recommendations_df['user_id'].nunique()
unique_games = recommendations_df['app_id'].nunique()
total_interactions = len(recommendations_df)
sparsity = 1 - (total_interactions / (unique_users * unique_games))

print(f"唯一用户数: {unique_users}")
print(f"唯一游戏数: {unique_games}")
print(f"总交互数: {total_interactions}")
print(f"数据稀疏度: {sparsity:.4f} ({sparsity*100:.2f}%)")

## 7. 数据预处理探索

In [ ]:
# 探索如何将游戏时长转换为评分

# 创建不同转换函数
recommendations_df['rating_linear'] = recommendations_df['hours'] / 10.0  # 线性转换
recommendations_df['rating_log'] = np.log1p(recommendations_df['hours'])  # 对数转换
recommendations_df['rating_hybrid'] = recommendations_df.apply(
    lambda row: min(10.0, row['hours'] / 10) * (1.5 if row['is_recommended'] else 0.5),
    axis=1
)  # 混合转换

# 可视化比较不同转换方法
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(recommendations_df['rating_linear'].clip(0, 10), bins=20)
plt.title('线性转换评分分布')
plt.xlabel('评分')
plt.xlim(0, 10)

plt.subplot(1, 3, 2)
sns.histplot(recommendations_df['rating_log'].clip(0, 10), bins=20)
plt.title('对数转换评分分布')
plt.xlabel('评分')
plt.xlim(0, 10)

plt.subplot(1, 3, 3)
sns.histplot(recommendations_df['rating_hybrid'].clip(0, 10), bins=20)
plt.title('混合转换评分分布')
plt.xlabel('评分')
plt.xlim(0, 10)

plt.tight_layout()
plt.show()

## 8. 总结与洞察

通过对Steam游戏数据集的探索，我们得到以下主要洞察：

1. **数据规模**：数据集包含了大量游戏、用户和评价信息，为构建推荐系统提供了丰富的基础。

2. **数据稀疏性**：用户-游戏交互矩阵非常稀疏，这是推荐系统常见的挑战。

3. **游戏特征**：游戏有丰富的元数据，包括描述和标签，可用于基于内容的推荐。

4. **评价特征**：用户评价包含游戏时长和是否推荐，可以结合这两个信息创建更准确的隐式反馈。

5. **用户行为**：用户拥有的游戏数量和评价数量分布不均，需要考虑处理长尾分布的策略。

基于以上发现，我们计划采用混合推荐策略，结合协同过滤和基于内容的方法，并特别关注冷启动问题的解决方案。